# Experiment 2.0



## Set up

In [2]:
notebook_name = 'experiment_2.0'



In [3]:
# Reload functions every time
%load_ext autoreload 
%autoreload 2

In [16]:
# Load libraries
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path

import json
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# Import the variables that have been set in the init.py folder in the root directory
# These include a constant called PROJECT_ROOT which stores the absolute path to this folder
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import init
PROJECT_ROOT = os.getenv("PROJECT_ROOT")

# Add the src folder to sys path, so that the application knows to look there for libraries
sys.path.append(str(Path(PROJECT_ROOT) / 'src'))






In [55]:
from functions.write_admission_note import write_admission_note

## Load data

Here the function called ...

In [36]:
FIHR_DIR = '../../privfp-poc-zk/experiments/02_generate_dataset'

import json

with open(FIHR_DIR + '/' + 'all_inpatient_visit_dict.json') as f:
   all_inpatient_visit_dict = json.load(f)
   
   # ed.head()


with open(FIHR_DIR + '/' + 'long_visit_dict.json') as f:
   long_visit_dict = json.load(f)

print(len(long_visit_dict.keys()))
   
ex = list(long_visit_dict.keys())[0]
ex =  "urn:uuid:6ac2bf6b-6a8c-51e9-a2dd-30eb780264c5"


18


## Generate output for ChatGPT

In [50]:

moment = datetime.datetime.fromisoformat(long_visit_dict[ex]['Encounter']['Encounter Started']) + datetime.timedelta(days = 1)
moment = moment.replace(tzinfo=None)  # Makes 'moment' naive by removing timezone information

print("Write an admission note for this patient")

print("The time now is " + str(moment))
print("The patient ID is " + long_visit_dict[ex]['Encounter']['Encounter id'])
print("The hospital visit began at " + str(long_visit_dict[ex]['Encounter']['Encounter Started']))
print("The type of admission is " + long_visit_dict[ex]['Encounter']['Type of admission'])
print("The patient has these disorders and symptons")
for key, value in long_visit_dict[ex]['Condition'].items():
    if datetime.datetime.fromisoformat(value['First Recorded']).replace(tzinfo=None) <= moment:
        print(value['Text'] + ': ' + 'First recorded ' + value['First Recorded'])
        # print(value['First Recorded'])

print('In the last 24 hours, the following observations have been recorded')
for key, value in long_visit_dict[ex]['Observation'].items():
    if datetime.datetime.fromisoformat(value['Recorded']).replace(tzinfo=None) <= moment:
        if 'Value' in value:
            print(value['Text'] + ': ' + str(value['Value']) + value['Units'] + '; Recorded ' + value['Recorded'])
        else:
            print(value['Text'] + '; Recorded ' + value['Recorded'])

        # print(value['First Recorded'])     
            

    




Write an admission note for this patient
The time now is 2021-01-08 04:51:40
The patient ID is urn:uuid:4d563c2d-50f5-a4d8-800d-d9566445e53c
The hospital visit began at 2021-01-07 04:51:40
The type of admission is Hospital admission for isolation (procedure)
The patient has these disorders and symptons
Pneumonia (disorder): First recorded 2021-01-07T05:34:26+00:00
Hypoxemia (disorder): First recorded 2021-01-07T05:34:26+00:00
Respiratory distress (finding): First recorded 2021-01-07T05:34:26+00:00
Acute respiratory failure (disorder): First recorded 2021-01-07T05:34:26+00:00
Sepsis caused by virus (disorder): First recorded 2021-01-07T05:34:26+00:00
In the last 24 hours, the following observations have been recorded
Body temperature: 41.724Cel; Recorded 2021-01-07T05:34:26+00:00
Respiratory rate: 12.329/min; Recorded 2021-01-07T05:34:26+00:00
Heart rate: 180.11/min; Recorded 2021-01-07T05:34:26+00:00
Oxygen saturation in Arterial blood: 82.39%; Recorded 2021-01-07T05:34:26+00:00
Blood 

In [34]:
!pip install jinja2

  Obtaining dependency information for jinja2 from https://files.pythonhosted.org/packages/30/6d/6de6be2d02603ab56e72997708809e8a5b0fbfee080735109b40a3564843/Jinja2-3.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for MarkupSafe>=2.0 from https://files.pythonhosted.org/packages/e9/58/e6de8fd932e62d7a43174e700290cba3da08a502955de22141fbfced4b42/MarkupSafe-2.1.4-cp311-cp311-macosx_10_9_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 3.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [52]:
import jinja2
import datetime


# Define the custom datetime filter for Jinja
def datetime_filter(value):
    return datetime.datetime.fromisoformat(value).replace(tzinfo=None)

# Set up the Jinja environment
env = jinja2.Environment()
env.filters['datetime'] = datetime_filter

# Define the template
template_str = """
{% macro format_admission_note(long_visit_dict, ex, moment) %}
    Write an admission note for this patient
    The time now is {{ moment }}
    Patient ID: {{ long_visit_dict[ex]['Encounter']['Encounter id'] }}
    Hospital visit began at: {{ long_visit_dict[ex]['Encounter']['Encounter Started'] }}
    Type of admission: {{ long_visit_dict[ex]['Encounter']['Type of admission'] }}
    Disorders and symptoms:
    {% for key, value in long_visit_dict[ex]['Condition'].items() %}
        {% if value['First Recorded'] <= moment.isoformat() %}
            {{ value['Text'] }}: First recorded on {{ value['First Recorded'] }}
        {% endif %}
    {% endfor %}
    Observations in the last 24 hours:
    {% for key, value in long_visit_dict[ex]['Observation'].items() %}
        {% if value['Recorded'] <= moment.isoformat() %}
            {{ value['Text'] }}; Recorded on {{ value['Recorded'] }}
            {% if 'Value' in value %}
                : {{ value['Value'] }}{{ value['Units'] }}
            {% endif %}
        {% endif %}
    {% endfor %}
{% endmacro %}
"""


# Load the template
template = env.from_string(template_str)

# Render the template
rendered = template.module.format_admission_note(long_visit_dict, ex, moment)
print(rendered)


    Write an admission note for this patient
    The time now is 2021-01-08 04:51:40
    Patient ID: urn:uuid:4d563c2d-50f5-a4d8-800d-d9566445e53c
    Hospital visit began at: 2021-01-07 04:51:40
    Type of admission: Hospital admission for isolation (procedure)
    Disorders and symptoms:
    
        
            Pneumonia (disorder): First recorded on 2021-01-07T05:34:26+00:00
        
    
        
            Hypoxemia (disorder): First recorded on 2021-01-07T05:34:26+00:00
        
    
        
            Respiratory distress (finding): First recorded on 2021-01-07T05:34:26+00:00
        
    
        
            Acute respiratory failure (disorder): First recorded on 2021-01-07T05:34:26+00:00
        
    
        
            Sepsis caused by virus (disorder): First recorded on 2021-01-07T05:34:26+00:00
        
    
    Observations in the last 24 hours:
    
        
            Body temperature; Recorded on 2021-01-07T05:34:26+00:00
            
                : 41.724

In [46]:
string = """Write an admission note for this patient
The time now is 2021-01-08 04:51:40
The patient ID is urn:uuid:4d563c2d-50f5-a4d8-800d-d9566445e53c
The hospital visit began at 2021-01-07 04:51:40
The type of admission is Hospital admission for isolation (procedure)
The patient has these disorders and symptons
Pneumonia (disorder): First recorded 2021-01-07T05:34:26+00:00
Hypoxemia (disorder): First recorded 2021-01-07T05:34:26+00:00
Respiratory distress (finding): First recorded 2021-01-07T05:34:26+00:00
Acute respiratory failure (disorder): First recorded 2021-01-07T05:34:26+00:00
Sepsis caused by virus (disorder): First recorded 2021-01-07T05:34:26+00:00
In the last 24 hours, the following observations have been recorded
Body temperature: 41.724Cel; Recorded 2021-01-07T05:34:26+00:00
Respiratory rate: 12.329/min; Recorded 2021-01-07T05:34:26+00:00
Heart rate: 180.11/min; Recorded 2021-01-07T05:34:26+00:00
Oxygen saturation in Arterial blood: 82.39%; Recorded 2021-01-07T05:34:26+00:00
Blood Pressure; Recorded 2021-01-07T05:34:26+00:00
Body Weight: 73.1kg; Recorded 2021-01-07T05:34:26+00:00
Leukocytes [#/volume] in Blood by Automated count: 3.963210*3/uL; Recorded 2021-01-07T05:34:26+00:00
Erythrocytes [#/volume] in Blood by Automated count: 5.77810*6/uL; Recorded 2021-01-07T05:34:26+00:00
Hemoglobin [Mass/volume] in Blood: 13.234g/dL; Recorded 2021-01-07T05:34:26+00:00
Hematocrit [Volume Fraction] of Blood by Automated count: 42.528%; Recorded 2021-01-07T05:34:26+00:00
MCV [Entitic volume] by Automated count: 95.64fL; Recorded 2021-01-07T05:34:26+00:00
MCH [Entitic mass] by Automated count: 31.824pg; Recorded 2021-01-07T05:34:26+00:00
MCHC [Mass/volume] by Automated count: 32.414g/dL; Recorded 2021-01-07T05:34:26+00:00
Erythrocyte distribution width [Ratio] by Automated count: 14.888%; Recorded 2021-01-07T05:34:26+00:00
Platelets [#/volume] in Blood by Automated count: 100.9210*3/uL; Recorded 2021-01-07T05:34:26+00:00
Neutrophils/100 leukocytes in Blood by Automated count: 38.21%; Recorded 2021-01-07T05:34:26+00:00
Lymphocytes/100 leukocytes in Blood by Automated count: 11.513%; Recorded 2021-01-07T05:34:26+00:00
Monocytes/100 leukocytes in Blood by Automated count: 14.377%; Recorded 2021-01-07T05:34:26+00:00
Eosinophils/100 leukocytes in Blood by Automated count: 3.8467%; Recorded 2021-01-07T05:34:26+00:00
Basophils/100 leukocytes in Blood by Automated count: 2.3733%; Recorded 2021-01-07T05:34:26+00:00
Neutrophils [#/volume] in Blood by Automated count: 3.488110*3/uL; Recorded 2021-01-07T05:34:26+00:00
Lymphocytes [#/volume] in Blood by Automated count: 0.7257510*3/uL; Recorded 2021-01-07T05:34:26+00:00
Monocytes [#/volume] in Blood by Automated count: 0.787810*3/uL; Recorded 2021-01-07T05:34:26+00:00
Eosinophils [#/volume] in Blood by Automated count: 0.5394610*3/uL; Recorded 2021-01-07T05:34:26+00:00
Basophils [#/volume] in Blood by Automated count: 0.2155710*3/uL; Recorded 2021-01-07T05:34:26+00:00
Glucose [Mass/volume] in Serum or Plasma: 149.62mg/dL; Recorded 2021-01-07T05:34:26+00:00
Urea nitrogen [Mass/volume] in Serum or Plasma: 8.13mg/dL; Recorded 2021-01-07T05:34:26+00:00
Creatinine [Mass/volume] in Serum or Plasma: 2.9042mg/dL; Recorded 2021-01-07T05:34:26+00:00
Calcium [Mass/volume] in Serum or Plasma: 9.32mg/dL; Recorded 2021-01-07T05:34:26+00:00
Sodium [Moles/volume] in Serum or Plasma: 143.1mmol/L; Recorded 2021-01-07T05:34:26+00:00
Potassium [Moles/volume] in Serum or Plasma: 4.81mmol/L; Recorded 2021-01-07T05:34:26+00:00
Chloride [Moles/volume] in Serum or Plasma: 109.16mmol/L; Recorded 2021-01-07T05:34:26+00:00
Carbon dioxide, total [Moles/volume] in Serum or Plasma: 24.82mmol/L; Recorded 2021-01-07T05:34:26+00:00
Glomerular filtration rate/1.73 sq M.predicted: 11.83mL/min; Recorded 2021-01-07T05:34:26+00:00
Protein [Mass/volume] in Serum or Plasma: 5.9069g/dL; Recorded 2021-01-07T05:34:26+00:00
Albumin [Mass/volume] in Serum or Plasma: 3.5103g/dL; Recorded 2021-01-07T05:34:26+00:00
Bilirubin.total [Mass/volume] in Serum or Plasma: 6.3625mg/dL; Recorded 2021-01-07T05:34:26+00:00
Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma: 81.719U/L; Recorded 2021-01-07T05:34:26+00:00
Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma: 33.073U/L; Recorded 2021-01-07T05:34:26+00:00
Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma: 34.791U/L; Recorded 2021-01-07T05:34:26+00:00
pH of Arterial blood: 7.265[pH]; Recorded 2021-01-07T05:34:26+00:00
Carbon dioxide [Partial pressure] in Arterial blood: 38.086mm[Hg]; Recorded 2021-01-07T05:34:26+00:00
Oxygen [Partial pressure] in Arterial blood: 38.142mm[Hg]; Recorded 2021-01-07T05:34:26+00:00
Bicarbonate [Moles/volume] in Arterial blood: 24.216mmol/L; Recorded 2021-01-07T05:34:26+00:00
Oxygen saturation in Arterial blood: 86.01%; Recorded 2021-01-07T05:34:26+00:00
Oxygen/Inspired gas setting [Volume Fraction] Ventilator: 95.009%; Recorded 2021-01-07T05:34:26+00:00
Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma: 1.9468ug/mL; Recorded 2021-01-07T05:34:26+00:00
Ferritin [Mass/volume] in Serum or Plasma: 828.88ug/L; Recorded 2021-01-07T05:34:26+00:00
Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method: 6.29pg/mL; Recorded 2021-01-07T05:34:26+00:00
Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction: 321.95U/L; Recorded 2021-01-07T05:34:26+00:00
Creatine kinase [Enzymatic activity/volume] in Serum or Plasma: 46.04U/L; Recorded 2021-01-07T05:34:26+00:00
C reactive protein [Mass/volume] in Serum or Plasma: 17.634mg/L; Recorded 2021-01-07T05:34:26+00:00
Prothrombin time (PT): 12.903s; Recorded 2021-01-07T05:34:26+00:00
INR in Platelet poor plasma by Coagulation assay: 4.6038{INR}; Recorded 2021-01-07T05:34:26+00:00
Procalcitonin [Mass/volume] in Serum or Plasma: 0.15391ng/mL; Recorded 2021-01-07T05:34:26+00:00
"""

In [40]:
!pip install tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/f4/99/90fefb027bd962cd336f9f955a2f27782aac123ee7943cfc8a7bd3c1f198/tiktoken-0.5.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/60/9e/4b0223e05776aa3be806a902093b2ab1de3ba26b652d92065d5c7e1d4df3/regex-2023.12.25-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 2.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [53]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")
# or "gpt-3.5-turbo" or "text-davinci-003"

tokens = encoding.encode(rendered)
token_count = len(tokens)
token_count

3008

In [56]:
write_admission_note(rendered)

("The patient presents with symptoms indicative of a severe viral infection including pneumonia, hypoxemia, and respiratory distress, notably linked to acute respiratory failure. The symptoms, along with a recorded body temperature of 41.724 Celsius degrees and a sepsis diagnosis suggest a potentially severe viral infection. There are also signs of critical metabolic imbalances, including elevated heart rate at 180.11 bpm, creatinine [Mass/volume] in Serum or Plasma at 2.9042 mg/dL and signified decrease in glomerular filtration rate at 11.83 mL/min, which is quite concerning. The patient's oxygen saturation levels in arterial blood are low at 82.39%, indicating his body isn't getting the oxygen it needs.",
 'Probable Severe Viral Infection with Acute Respiratory Failure',
 '3-4 weeks')